Source: https://www.kaggle.com/ahassaine/pure-image-processing-lb-0-274/code 


# Libraries and Global Parameters 

In [19]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.util import img_as_bool, img_as_uint, img_as_ubyte
from skimage.transform import resize
#import skimage
#import glob
import random
from random import randint #, shuffle

from keras import regularizers
from keras.models import Model, load_model
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Activation, Dense, \
    UpSampling2D, BatchNormalization, add, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras import backend as K
from keras.losses import binary_crossentropy, sparse_categorical_crossentropy

model_checkpoint_file='meshnet_v2.h5'

# Root folders for test and training data
train_root = "./stage1_train"
test_root = "./stage1_test"

# Size we resize all images to
#image_size = (128,128)

img_height = 128
img_width = 128

import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='skimage')

# Preparing the Data

In [6]:
## Import Training Data Images 
train_dirs = os.listdir(train_root)
train_filenames=[os.path.join(train_root,file_id) + "/images/"+file_id+".png" for file_id in train_dirs]
# Convert to B&W inline
#train_images=[cv2.cvtColor(cv2.imread(imagefile),cv2.COLOR_BGR2GRAY) for imagefile in train_filenames]
train_images=[imread(imagefile,as_grey=True) for imagefile in train_filenames]

# Use this instead if you want color images
#train_images=[imread(imagefile) for imagefile in train_filenames]


In [7]:
## Import Training Masks
# this takes longer than the training images because we have to
# combine a lot of mask files

# This function creates a single combined mask image 
# when given a list of masks
# Probably a computationally faster way to do this...
def collapse_masks(mask_list):
    for i, mask_file in enumerate(mask_list):        
        if i != 0:
            # combine mask with previous mask in list 
            mask = np.maximum(mask, imread(os.path.join(train_root,mask_file)))
        else:
            # read first mask in
            mask = imread(os.path.join(train_root,mask_file))
    
    return img_as_ubyte(mask)

# Import all the masks
train_mask_dirs = [ os.path.join(path, 'masks') for path in os.listdir(train_root) ]
train_mask_files = [ [os.path.join(dir,file) for file in os.listdir(os.path.join(train_root,dir)) ]  for dir in train_mask_dirs]

#def collapse_masks(mask_list):
#    for i, mask_file in enumerate(mask_list):
#        print(i)
#        print(mask_file)
        
#testing = [collapse_masks(mask_files) for mask_files in train_mask_files]

# Divide output of each mask by 255 to make a 1/0 binary mask
train_masks = [ collapse_masks(mask_files) for mask_files in train_mask_files ]


# Computer Vision Technique

In [14]:
def comp_viz_mask(img):
    #green channel happends to produce slightly better results
    #than the grayscale image and other channels
   # img_gray=img_rgb[:,:,1]#cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    #morphological opening (size tuned on training data)
    circle7=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7))
    img_open=cv2.morphologyEx(img, cv2.MORPH_OPEN, circle7)
    #Otsu thresholding
    img_th=cv2.threshold(img_open,0,255,cv2.THRESH_OTSU)[1]
    #Invert the image in case the objects of interest are in the dark side
    if(np.sum(img_th==255)>np.sum(img_th==0)):
        img_th=cv2.bitwise_not(img_th)
    #second morphological opening (on binary image this time)
    bin_open=cv2.morphologyEx(img_th, cv2.MORPH_OPEN, circle7) 
    #connected components
    cc=cv2.connectedComponents(bin_open)[1]
    #cc=segment_on_dt(bin_open,20)
    return cc # convert to 1/0

In [ ]:
train_cv2_masks[0].dtype

In [ ]:
train_masks[0].max()

In [17]:
# Plot images side by side for a list of datasets
def plot_side_by_side(ds_list,image_num):
    #print('Image #: ' + str(image_num) + '. Image Sizes: ' + str(image_ds[image_num].shape) + ' ' + str(mask_ds[image_num].shape))
    fig = plt.figure(figsize=(20,10))

    for i in range(len(ds_list)):
        ax1 = fig.add_subplot(1,len(ds_list),i+1)
        ax1.imshow(ds_list[i][image_num])

    plt.show()
    
# Plots random corresponding images and masks
def plot_check(ds_list,rand_imgs=None,img_nums=None):
    if rand_imgs != None:
        for i in range(rand_imgs):
            plot_side_by_side(ds_list, randint(0,len(ds_list[0])-1))
    if img_nums != None:
        for i in range(len(img_nums)):
            plot_side_by_side(ds_list,img_nums[i])
    
#plot_side_by_side(train_images,train_mask_images,38)
# Plot a few random images 
#plot_check(train_images,train_masks,rand_imgs=3)
plot_check([train_images,train_cv2_masks,train_masks],rand_imgs=3)
#plot_check(train_images,cv2_masks,rand_imgs=3)

#plot_check(train_images,train_mask_images,img_nums=[309])



NameError: name 'train_cv2_masks' is not defined

In [18]:
# Resize everything
# Also do dtype conversions

# Scaling
resized_train_images = [ img_as_ubyte(resize(image,(img_width,img_height))) for image in train_images] 

resized_train_masks = [ img_as_bool(resize(image,(img_width,img_height))) for image in train_masks] 
resized_train_cv2_masks = [ img_as_bool(resize(image,(img_width,img_height))) for image in train_cv2_masks] 

#Croping

#crop_size=64

#resized_train_images = [ image[int(0.5*(image.shape[0]-crop_size)):int(0.5*(image.shape[0]+crop_size)),
#   int(0.5*(image.shape[1]-crop_size)):int(0.5*(image.shape[1]+crop_size))] for image in train_images] 

#resized_train_mask_images = [ image[int(0.5*(image.shape[0]-crop_size)):int(0.5*(image.shape[0]+crop_size)),
#   int(0.5*(image.shape[1]-crop_size)):int(0.5*(image.shape[1]+crop_size))] for image in train_mask_images] 


NameError: name 'train_cv2_masks' is not defined

In [ ]:
# check max pixel values
print(resized_train_images[309].max())
print(resized_train_images[16].max())

In [13]:
# Check to see that my resizing and compression is screwing stuff up
rand_img_num = randint(0,len(train_images))
#plot_check(compressed_train_images,compressed_train_masks,img_nums=[rand_img_num])
#plot_check(train_images,train_mask_images,img_nums=[rand_img_num])

print('Original image and mask')
plot_check([train_images,train_cv2_masks,train_masks],img_nums=[492])
print('Resized image and mask')
plot_check([resized_train_images,resized_train_cv2_masks,resized_train_masks],img_nums=[492])

#plot_check(train_images,train_mask_images,img_nums=[664])
#print(train_images[664].max())

Original image and mask


NameError: name 'train_cv2_masks' is not defined

In [4]:
# Check to see that my resizing and compression is screwing stuff up
rand_img_num = randint(0,len(train_images))
#plot_check(compressed_train_images,compressed_train_masks,img_nums=[rand_img_num])
#plot_check(train_images,train_mask_images,img_nums=[rand_img_num])

print('Original image and mask')
plot_check([train_images,train_masks],img_nums=[492])
print('Resized image and mask')
plot_check([resized_train_images,resized_train_masks],img_nums=[492])

#plot_check(train_images,train_mask_images,img_nums=[664])
#print(train_images[664].max())

NameError: name 'train_images' is not defined

In [20]:
# Reshape model inputs
train_X = np.reshape(np.array(resized_train_images),(len(resized_train_images),img_height,img_width,1))

# Stack cv2 masks on top of images as a channel
#train_X = np.reshape(np.stack((np.array(resized_train_images),np.array(resized_train_cv2_masks)),axis=3), \
#                     (len(resized_train_images),img_height,img_width,2))
train_Y = np.reshape(np.array(resized_train_masks),(len(resized_train_masks),img_height,img_width,1))

# Check size of arrays we are inputting to model
# This is important! We need the datasets to be as 
# small as possible to reduce computation time
print(train_X.shape)
print(train_Y.shape)
print(train_X.nbytes)
print(train_Y.nbytes)

(670, 128, 128, 1)
(670, 128, 128, 1)
10977280
10977280


In [22]:
# Check datatypes
print(train_Y.dtype)
print(train_X.dtype)

#train_X[0]

bool
uint8


In [1]:
#plot_check([np.squeeze(train_X,axis=3),np.squeeze(train_Y,axis=3)],img_nums=[5,119])
#plot_check([train_images,train_masks],img_nums=[5,119])

In [31]:
train_X[602].max() # check max pixel value

255

# Now Let's Build the Model

In [32]:
# Loss and metric functions for the neural net
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred = K.cast(y_pred, 'float32')
    y_pred_f = K.cast(K.greater(K.flatten(y_pred), 0.5), 'float32')
    intersection = y_true_f * y_pred_f
    score = 2. * K.sum(intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))
    return score

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

def create_block(x, filters=21, filter_size=(3, 3), activation='relu',dil_rate=1,dropout_rate=0.2):
#    for i in range(n_block):
    x = Conv2D(filters, filter_size, padding='same', activation='relu', dilation_rate = dil_rate) (x)
    #x = Activation('relu') (x)
    x = BatchNormalization() (x) 
    x = Dropout(dropout_rate) (x)
    return x


## master function for creating a net
def get_net(
        input_shape=(img_height, img_width,1),
        loss=binary_crossentropy,
        n_class=1
):
    inputs = Input(input_shape)
    
    # Create layers
    net_body = create_block(inputs)
    net_body = create_block(net_body)
    net_body = create_block(net_body)
    net_body = create_block(net_body,dil_rate=2)
    net_body = create_block(net_body,dil_rate=4)
    net_body = create_block(net_body,dil_rate=8)
    net_body = create_block(net_body)
    classify = Conv2D(n_class,(1,1),activation='sigmoid') (net_body)
    
    #classify = Activation(activation='sigmoid') (net_body)
    #classify = Dense(1,activation='sigmoid') (net_body)

    model = Model(inputs=inputs, outputs=classify)
    model.compile(optimizer=Adam(0.001), loss=loss, metrics=[bce_dice_loss, dice_coef])

    return model

In [33]:
my_model = get_net()

In [34]:
print(my_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 128, 128, 1)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 128, 128, 21)      210       
_________________________________________________________________
batch_normalization_15 (Batc (None, 128, 128, 21)      84        
_________________________________________________________________
dropout_15 (Dropout)         (None, 128, 128, 21)      0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 128, 128, 21)      3990      
_________________________________________________________________
batch_normalization_16 (Batc (None, 128, 128, 21)      84        
_________________________________________________________________
dropout_16 (Dropout)         (None, 128, 128, 21)      0         
__________

In [ ]:
# Fit model

# learning rate decay https://keras.io/callbacks/#learningratescheduler 

# This function controls the learning rate based on the epoch
# must return a float
    
#change_lr = LearningRateScheduler(scheduler,verbose=1)
earlystopper = EarlyStopping(patience=10, verbose=1)
checkpointer = ModelCheckpoint(model_checkpoint_file, verbose=1, save_best_only=True)
reduce_plateau = ReduceLROnPlateau(monitor='val_loss', 
                           factor=0.5,
                           patience=3,
                           verbose=1,
                        #   min_lr=0.00001,
                            epsilon=0.001,
                           mode='auto') 
# verbose=2 => one line per epoch, 1 = progress bar

### Add an epsilon 
results = my_model.fit(train_X, train_Y, validation_split=0.1, batch_size=10, epochs=75, verbose=1, 
                       shuffle=True, callbacks=[ earlystopper, checkpointer, reduce_plateau])

Train on 603 samples, validate on 67 samples
Epoch 1/75
600/603 [============================>.] - ETA: 1s - loss: 0.6506 - bce_dice_loss: 1.3553 - dice_coef: 0.3966
Epoch 00001: val_loss improved from inf to 0.37011, saving model to meshnet_v2.h5
603/603 [==============================] - 357s 592ms/step - loss: 0.6502 - bce_dice_loss: 1.3553 - dice_coef: 0.3963 - val_loss: 0.3701 - val_bce_dice_loss: 0.9163 - val_dice_coef: 0.6410
Epoch 2/75
600/603 [============================>.] - ETA: 1s - loss: 0.3764 - bce_dice_loss: 0.9238 - dice_coef: 0.6662
Epoch 00002: val_loss improved from 0.37011 to 0.22300, saving model to meshnet_v2.h5
603/603 [==============================] - 348s 578ms/step - loss: 0.3771 - bce_dice_loss: 0.9250 - dice_coef: 0.6650 - val_loss: 0.2230 - val_bce_dice_loss: 0.6370 - val_dice_coef: 0.7585
Epoch 3/75
600/603 [============================>.] - ETA: 1s - loss: 0.2554 - bce_dice_loss: 0.7045 - dice_coef: 0.7662
Epoch 00003: val_loss improved from 0.22300 to

In [ ]:
#Epoch 00013: LearningRateScheduler reducing learning rate to 0.0001.
#Epoch 13/75
#600/603 [============================>.] - ETA: 1s - loss: 0.2774 - dice_coef: 0.8625 - binary_crossentropy: 0.1149
#Epoch 00013: val_loss improved from 0.25185 to 0.24854, saving model to bw_model_v8.h5
#603/603 [==============================] - 221s 367ms/step - loss: 0.2768 - dice_coef: 0.8628 - binary_crossentropy: 0.1146 - val_loss: 0.2485 - val_dice_coef: 0.8691 - val_binary_crossentropy: 0.1000
#Epoch 00017: LearningRateScheduler reducing learning rate to 0.0001.
#Epoch 17/75
#600/603 [============================>.] - ETA: 1s - loss: 0.2818 - dice_coef: 0.8604 - binary_crossentropy: 0.1164
#Epoch 00017: val_loss did not improve
#603/603 [==============================] - 217s 360ms/step - loss: 0.2842 - dice_coef: 0.8593 - binary_crossentropy: 0.1176 - val_loss: 0.2480 - val_dice_coef: 0.8686 - val_binary_crossentropy: 0.1002


# Make predictions
final_model = load_model(model_checkpoint_file, custom_objects={'dice_coef': dice_coef, 'bce_dice_loss':bce_dice_loss})
#preds_train = final_model.predict(train_X[:int(train_X.shape[0]*0.9)], verbose=1)
#preds_val = final_model.predict(train_X[int(train_X.shape[0]*0.9):], verbose=1)

preds_train = final_model.predict(train_X, verbose=1)

preds_train_t = (preds_train > 0.5).astype(np.uint8)
#preds_val_t = (preds_val > 0.5).astype(np.uint8)

In [ ]:
Ideas
 - Experiment with compression of training data. Am I preserving as much detail
    as I can in dtype np.uint8 (values of 0 to 255) ?
- Color vs B&W?
- Combine mask and prediction images to show false positives and negatives
- What is the best resizing method? Reflect??
- Put computer vision / threshold method output as an input to neural net
- Output intermediate layers for inspection
- Crop images to train networks faster for testing ??
- Take random crops of images to create, and then combine outputs in the end
- Is combining the masks really the best thing to do? Should I be keeping the individual cells separate?
- Pseudo-labelled data